In [1]:
from tkinter import Scrollbar, Button, Canvas, messagebox
from openai import OpenAI

import os
import pygame
import threading
import tempfile
import sqlite3
import math
import tkinter as tk
import sounddevice as sd
import scipy.io.wavfile as wavfile

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# OpenAI 클라이언트 초기화
client = OpenAI()

In [3]:
# Pygame 초기화
pygame.init()
pygame.mixer.init()

In [4]:
class WordDatabase:
    def __init__(self, db_path, splited_text, client):
        """
        :param db_path: 데이터베이스 파일 경로
        :param splited_text: 마침표 단위로 분할된 텍스트 목록
        :param client: OpenAI API 클라이언트 인스턴스
        """
        self.db_path = db_path
        self.splited_text = splited_text
        self.client = client
        self.initialize_database()

    def initialize_database(self):
        if not os.path.exists(self.db_path):
            print("Database not found. Creating new database and populating word explanations...")
            self.create_database()
        else:
            print("Database already exists. Skipping creation.")

    def create_database(self):
        # SQLite 연결 및 테이블 생성
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS word_explanations (
                word TEXT PRIMARY KEY,
                audio_file TEXT,  # 새로운 컬럼 추가
                explanation TEXT
            )
        ''')
        conn.commit()
        
        # 전체 텍스트를 단어 단위로 분할
        words = self.extract_unique_words(self.full_text())
        print(f"Extracted {len(words)} unique words.")
        
        # 단어 목록을 배치 단위로 분할
        batches = self.split_into_batches(words, batch_size=50)  # 배치 크기 조정 가능
        print(f"Processing {len(batches)} batches of words.")
        
        # 각 배치에 대해 설명을 요청하고 데이터베이스에 저장
        for i, batch in enumerate(batches, 1):
            print(f"Processing batch {i}/{len(batches)}: {len(batch)} words")
            explanations = self.get_word_explanations_batch(batch)
            audio_files = self.generate_audio_files_batch(batch)
            for word in batch:
                explanation = explanations.get(word.lower(), "설명을 가져오는 데 실패했습니다.")
                audio_file = audio_files.get(word.lower(), None)
                cursor.execute('''
                    INSERT INTO word_explanations (word, audio_file, explanation) VALUES (?, ?, ?)
                ''', (word.lower(), audio_file, explanation))
            conn.commit()
            print(f"Batch {i} processed and saved.")
        
        conn.close()
        print("Database creation and population completed.")

    def full_text(self):
        """
        전체 텍스트를 하나의 문자열로 반환합니다.
        """
        return ".".join(self.splited_text)

    def split_into_batches(self, words, batch_size=50):
        """
        단어 목록을 배치 단위로 분할합니다.
        :param words: 단어 목록 (리스트)
        :param batch_size: 한 배치당 단어 수
        :return: 배치 단어 목록 (리스트의 리스트)
        """
        words = list(words)
        num_batches = math.ceil(len(words) / batch_size)
        return [words[i * batch_size:(i + 1) * batch_size] for i in range(num_batches)]

    def extract_unique_words(self, text):
        """
        텍스트에서 고유한 단어를 추출합니다.
        :param text: 전체 텍스트 문자열
        :return: 고유 단어 집합
        """
        korean_punctuations = '.,!?~@#$%^&*()_+-={}|[]:;"\'<>?/\\'
        translator = str.maketrans('', '', korean_punctuations)
        words = set()
        for sentence in self.splited_text:
            # 단어 단위로 분할하고 불필요한 문자 제거
            for word in sentence.split():
                clean_word = word.translate(translator).strip().lower()
                if clean_word:
                    words.add(clean_word)
        return words

    def get_word_explanations_batch(self, words_batch):
        """
        배치 단위로 단어 설명을 요청합니다.
        :param words_batch: 단어 목록 (리스트)
        :return: 단어와 설명의 딕셔너리
        """
        try:
            # 프롬프트 구성: 각 단어에 대한 설명을 요청
            prompt = "다음 한국어 단어들의 간단한 설명을 제공해 주세요. 각 단어과 설명은 '단어: 설명' 형식으로 작성해 주세요.\n\n"
            for word in words_batch:
                prompt += f"{word}\n"
            
            response = self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt},
                ],
                max_tokens=1500  # 배치 크기에 따라 조정
            )
            explanations_text = response.choices[0].message.content.strip()
            
            # 응답 파싱: '단어: 설명' 형식으로 가정
            explanations = {}
            for line in explanations_text.split('\n'):
                if ':' in line:
                    word, explanation = line.split(':', 1)
                    explanations[word.strip().lower()] = explanation.strip()
            
            return explanations
        except Exception as e:
            print(f"Error fetching explanations for batch: {e}")
            # 실패한 단어에 대한 기본 설명 반환
            return {word.lower(): "설명을 가져오는 데 실패했습니다." for word in words_batch}

    def lookup_word_explanation(self, word):
        """
        데이터베이스에서 단어의 설명을 조회합니다.
        :param word: 조회할 단어
        :return: 단어의 설명 또는 None
        """
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('SELECT explanation FROM word_explanations WHERE word = ?', (word.lower(),))
        result = cursor.fetchone()
        conn.close()
        if result:
            return result[0]
        else:
            return None


In [5]:
class HighlightingApp:
    def __init__(self, root, input_text):
        print("Initializing HighlightingApp...")
        self.root = root
        self.splited_text = [sentence.strip() for sentence in input_text.split('.') if sentence.strip()]
        print(f"Split text into {len(self.splited_text)} sentences.")
        self.total_pages = len(self.splited_text) + 2  # 마지막 전체 텍스트 페이지 + 문제 페이지 포함
        self.current_page = 0
        # 페이지 프레임을 저장하기 위한 변수
        self.page_frames = []
        # 음성 파일 존재 여부를 관리하기 위한 변수
        self.audio_files = [f"audio_{i}.mp3" for i in range(len(self.splited_text))]
        self.audio_files.append("audio_full.mp3")  # 전체 문장을 위한 추가 음성 파일
        # 문제 저장 변수
        self.generated_question = ""
        # 전체 텍스트 저장
        self.full_text = ".".join(self.splited_text)

        # WordDatabase 인스턴스 생성
        self.word_db = WordDatabase(db_path="word_explanations.db", splited_text=self.splited_text, client=client)

        # 음성 파일 생성
        self.generate_audio_files()
        # 문제 생성
        self.generate_question()
        # GUI 초기화
        self.init_gui()

    def lookup_word_explanation(self, word):
        """
        WordDatabase 인스턴스를 사용하여 단어 설명을 조회합니다.
        """
        return self.word_db.lookup_word_explanation(word)


    def init_gui(self):
        print("Initializing GUI...")
        self.root.title("텍스트 하이라이팅")
        self.root.geometry("1000x800")

        # 스크롤 가능한 캔버스 생성
        canvas = Canvas(self.root)
        scrollbar = Scrollbar(self.root, orient="vertical", command=canvas.yview)
        scrollable_frame = tk.Frame(canvas)

        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )

        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)

        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")

        # 각 페이지를 위한 프레임 생성
        for i in range(self.total_pages):
            frame = tk.Frame(scrollable_frame)
            self.page_frames.append(frame)
            print(f"Created frame for page {i}.")

        # 첫 페이지로 이동
        print("Navigating to the first page.")
        self.show_page(0)

    def generate_audio_files(self):
        for i, sentence in enumerate(self.splited_text):
            self.create_audio_if_not_exists(sentence, self.audio_files[i])
        # 전체 텍스트에 대한 음성 파일 생성
        self.create_audio_if_not_exists(self.full_text, self.audio_files[-1])

    def create_audio_if_not_exists(self, text, audio_file):
        if not os.path.exists(audio_file):
            print(f"Generating audio for: {text[:30]}...")
            self.create_audio_from_text(text, audio_file)
        else:
            print(f"Audio file already exists: {audio_file}")

    def create_audio_from_text(self, input_text, output_file):
        print(f"Creating audio from text: {input_text[:30]}...")
        response = client.audio.speech.create(
            model="tts-1",
            voice="alloy",
            input=input_text,
        )
        with open(output_file, "wb") as f:
            f.write(response.content)
        print(f"음성 파일이 '{output_file}'로 저장되었습니다.")

    def play_audio(self, audio_file):
        print(f"Playing audio file: {audio_file}")
        pygame.mixer.music.load(audio_file)
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            pygame.time.Clock().tick(10)

    def generate_question(self):
        print("Generating question based on the full text...")
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"다음 텍스트를 바탕으로 간단한 질문 하나를 만들어 주세요: {self.full_text}"},
            ],
        )
        self.generated_question = response.choices[0].message.content
        print(f"Generated question: {self.generated_question}")

    def show_page(self, page_number):
        print(f"Attempting to show page {page_number}...")
        if page_number < 0 or page_number >= self.total_pages:
            print(f"Page {page_number} is out of range. Total pages: {self.total_pages}.")
            return

        # 현재 페이지 숨기기
        if hasattr(self, 'current_frame'):
            print(f"Hiding current frame for page {self.current_page}.")
            self.current_frame.pack_forget()

        # 새로운 페이지 프레임 활성화
        self.current_frame = self.page_frames[page_number]
        print(f"Activating frame for page {page_number}.")

        if page_number == self.total_pages - 2:
            self.setup_full_text_page()
        elif page_number == self.total_pages - 1:
            self.setup_question_page()
        else:
            self.setup_sentence_page(page_number)

        self.current_frame.pack(fill="both", expand=True)
        self.current_page = page_number
        print(f"Switched to page {page_number}")

        if page_number < self.total_pages - 1:
            threading.Timer(0.5, lambda: self.play_audio(self.audio_files[page_number])).start()

    def setup_sentence_page(self, page_number):
        text_to_display = self.splited_text[page_number]
        print(f"Displaying sentence for page {page_number}: {text_to_display[:30]}...")

        self.add_text_widget(self.current_frame, text_to_display)
        self.add_navigation_buttons(self.current_frame, page_number)

    def setup_full_text_page(self):
        print("Displaying full text on the second to last page.")
        self.add_text_widget(self.current_frame, self.full_text)
        self.add_navigation_buttons(self.current_frame, self.total_pages - 2, full_text=True)

    def setup_question_page(self):
        print("Setting up question page...")
        question_label = tk.Label(self.current_frame, text=self.generated_question, font=("Helvetica", 16), wraplength=700)
        question_label.pack(pady=10)
        print("Question added to the page.")

        answer_entry = tk.Entry(self.current_frame, width=80, font=("Helvetica", 14))
        answer_entry.pack(pady=10)
        print("Answer entry added to the page.")

        def check_answer():
            user_answer = answer_entry.get()
            print(f"User answer: {user_answer}")
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": f"다음 전체 텍스트를 참고하여, 주어진 질문에 대한 내 답변이 올바른지 검토해 주세요.\n\n전체 텍스트: {self.full_text}\n\n질문: '{self.generated_question}'\n\n나의 답변: '{user_answer}'"},
                ],
            )
            feedback = response.choices[0].message.content
            print(f"Feedback: {feedback}")
            feedback_label.config(text=feedback)

        submit_button = Button(self.current_frame, text="제출", command=check_answer)
        submit_button.pack(pady=5)
        print("Submit button added to the page.")

        feedback_label = tk.Label(self.current_frame, text="", font=("Helvetica", 14), wraplength=700)
        feedback_label.pack(pady=10)
        print("Feedback label added to the page.")
    
    def add_text_widget(self, parent, text):
        text_widget = tk.Text(parent, wrap="word", font=("Helvetica", 16))
        text_widget.pack(expand=True, fill="both")
        text_widget.insert("1.0", text)
        text_widget.configure(state="disabled")
        print("Text widget created and text inserted.")
    
        # 더블 클릭 이벤트 바인딩 추가
        text_widget.configure(state="normal")  # 바인딩 전에 상태를 변경
        text_widget.bind("<Double-1>", self.on_double_click)
        text_widget.configure(state="disabled")  # 다시 비활성화
        print("Double-click event bound to text widget.")


    def on_double_click(self, event):
        widget = event.widget
        try:
            # 클릭한 위치의 인덱스 가져오기
            index = widget.index(f"@{event.x},{event.y}")

            # 단어의 시작과 끝 인덱스 계산 using wordstart and wordend
            word_start = widget.index(f"{index} wordstart")
            word_end = widget.index(f"{index} wordend")

            if not word_start or not word_end:
                return  # 단어를 찾지 못한 경우 종료

            # 단어 추출
            word = widget.get(word_start, word_end).strip()
            
            # 디버깅을 위한 선택된 단어 출력
            print(f"Selected word: {word}")

            if word:
                # 팝업 창에 단어 설명 표시
                self.show_popup(word)

        except Exception as e:
            print(f"Error in on_double_click: {e}")


    def show_popup(self, word):
        explanation = self.lookup_word_explanation(word)
        if explanation is None:
            explanation = "설명이 없습니다."
        
        popup = tk.Toplevel()
        popup.title(f"'{word}'의 설명")
        popup.geometry("400x200")
    
        word_label = tk.Label(popup, text=word, font=("Helvetica", 14, "bold"))
        word_label.pack(pady=10)
    
        explanation_text = tk.Text(popup, wrap="word", font=("Helvetica", 12))
        explanation_text.pack(expand=True, fill="both", padx=10, pady=10)
        explanation_text.insert("1.0", explanation)
        explanation_text.configure(state="disabled")  # 편집 불가 상태로 설정
    
        close_button = tk.Button(popup, text="닫기", command=popup.destroy)
        close_button.pack(pady=5)


    def add_navigation_buttons(self, parent, page_number, full_text=False):
        button_frame = tk.Frame(parent)
        button_frame.pack()
        print("Button frame created.")

        if page_number > 0:
            prev_button = Button(button_frame, text="이전 페이지", command=lambda: self.show_page(page_number - 1))
            prev_button.pack(side="left", padx=5, pady=5)
            print("Previous button added.")

        if not full_text:
            next_button = Button(button_frame, text="다음 페이지", command=lambda: self.show_page(page_number + 1))
            next_button.pack(side="right", padx=5, pady=5)
            print("Next button added.")

        if full_text:
            next_button = Button(button_frame, text="다음 페이지", command=lambda: self.show_page(self.total_pages - 1))
            next_button.pack(side="right", padx=5, pady=5)
            print("Next button added.")

        if not full_text:
            tts_button = Button(button_frame, text="음성 재생", command=lambda: self.play_audio(self.audio_files[page_number]))
        else:
            tts_button = Button(button_frame, text="음성 재생", command=lambda: self.play_audio(self.audio_files[-1]))
        tts_button.pack(side="right", padx=5, pady=5)
        print("TTS button added.")

    def handle_user_pronunciation(self, page_number):
        print(f"Handling user pronunciation for page {page_number}...")
        original_text = self.splited_text[page_number]
        recorded_file = self.record_user_audio()
        transcribed_text = self.transcribe_audio(recorded_file)
        similarity_score = self.compare_texts(original_text, transcribed_text)
        print(f"User pronunciation similarity score: {similarity_score:.2f}%")
        messagebox.showinfo("발음 평가 결과", f"발음 유사도: {similarity_score:.2f}%")

    def record_user_audio(self, duration=5):
        print(f"Recording audio for {duration} seconds...")
        fs = 44100  # 샘플링 주파수
        recording = sd.rec(int(duration * fs), samplerate=fs, channels=2, dtype='int16')
        sd.wait()  # 녹음이 끝날 때까지 대기
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.wav')
        wavfile.write(temp_file.name, fs, recording)
        print(f"Audio recorded and saved to {temp_file.name}")
        return temp_file.name

    def transcribe_audio(self, audio_file):
        print(f"Transcribing audio file: {audio_file}")
        with open(audio_file, "rb") as audio:
            transcription = client.audio.transcriptions.create(
                model="whisper-1",
                file=audio,
                response_format="text"
            )
        transcribed_text = transcription
        print(f"Transcription result: {transcribed_text}")
        return transcribed_text

    def compare_texts(self, original_text, transcribed_text):
        original_words = set(original_text.lower().split())
        transcribed_words = set(transcribed_text.lower().split())
        common_words = original_words.intersection(transcribed_words)
        similarity = len(common_words) / len(original_words) if original_words else 0
        print(f"Similarity score: {similarity * 100:.2f}%")
        return similarity * 100



In [ ]:
orig_text = """레이첼 린드 여사는 자신의 관심사와 다른 사람의 염려를 협상으로 관리할 수 있는 능력이 있었다. 그녀는 봉제 동그라미를 그렸고, 주일 학교를 운영하는 것을 도왔으며 교회 원조 사회와 외무부 보조에서 가장 강력한 소장이었다. 그러나 레이첼 여사는 부엌 창문에서 몇 시간 동안 앉아서 '면 워프' 퀼트를 뜨개질하는 시간을 많이 보냈다."""
print("Starting application...")
root = tk.Tk()
app = HighlightingApp(root, orig_text)
print("Running main loop...")
root.mainloop()
print("Application closed.")

Starting application...
Initializing HighlightingApp...
Split text into 3 sentences.
Database already exists. Skipping creation.
Audio file already exists: audio_0.mp3
Audio file already exists: audio_1.mp3
Audio file already exists: audio_2.mp3
Audio file already exists: audio_full.mp3
Generating question based on the full text...
Generated question: 레이첼 린드 여사는 어떤 취미 활동을 즐겼나요?
Initializing GUI...
Created frame for page 0.
Created frame for page 1.
Created frame for page 2.
Created frame for page 3.
Created frame for page 4.
Navigating to the first page.
Attempting to show page 0...
Activating frame for page 0.
Displaying sentence for page 0: 레이첼 린드 여사는 자신의 관심사와 다른 사람의 염려를...
Text widget created and text inserted.
Double-click event bound to text widget.
Button frame created.
Next button added.
TTS button added.
Switched to page 0
Running main loop...
Playing audio file: audio_0.mp3
Attempting to show page 1...
Hiding current frame for page 0.
Activating frame for page 1.
Displaying se

: 